In [ ]:

!pip install transformers
!pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade
!pip install pytorch-lightning

In [ ]:
import pandas as pd
import re
import sklearn
import nltk
from sklearn.model_selection import train_test_split
from google.colab import drive


from transformers import RobertaTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl

In [ ]:
# Mount the Google Drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Read fake news and real news datasets

fake_news = pd.read_csv("gdrive/MyDrive/fake_news.csv")
real_news = pd.read_csv("gdrive/MyDrive/real_news.csv")
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
# Fake news data gets a label of 1 and real news data gets a label of 0

fake_news["label"] = 1
real_news["label"] = 0
data = pd.concat([fake_news, real_news], axis=0) # Concatenate both the dataframes

In [ ]:
# Randomly shuffle the concatenated dataframe

data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
# Take only the text and label columns

data = data[["text", "label"]]
data.head()

,text,label
0,"MEXICO CITY (Reuters) - At first, Mexico’s gov...",0
1,In their pathetic attempt to pass the deeply u...,1
2,Donald Trump s disastrous decision to take the...,1
3,"(Reuters) - Several countries, the United Nati...",0
4,,1


In [ ]:
# Clean the text

nltk.download("stopwords")
def clean_text(text):
  stopwords = nltk.corpus.stopwords.words('english')
  text = text.lower() # Convert to lower case
  text = re.sub(r'[^\w\s]', '', text) # Remove everything except words
  words = [word for word in text.split() if word not in stopwords] # Remove stopwords
  text = " ".join(words)
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data["text"] = data["text"].apply(clean_text)

In [ ]:
x = data["text"].values
y = data["label"].values

# Split into training and validation sets

train_data, val_data, train_labels, val_labels = train_test_split(x, y)

Training tokenizer

In [ ]:
# Write the list of sentences into a text file

with open("gdrive/MyDrive/data.txt", "w") as f:
  f.write("\n".join(data["text"].tolist()))

In [ ]:
# Train a BPE tokenizer from scratch on the data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

files = ["gdrive/MyDrive/data.txt"]
tokenizer.train(files, trainer)

In [ ]:
# Load the pretrained tokenizer and add PAD token 

from transformers import PreTrainedTokenizerFast
trained_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
trained_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

Using a Pretrained Tokenizer

In [ ]:
# Tokenize the articles

train_tokens = trained_tokenizer(list(train_data), return_tensors="pt", padding=True, truncation=True, max_length=64)
val_tokens = trained_tokenizer(list(val_data), return_tensors="pt", padding=True, truncation=True, max_length=64)

In [ ]:
# Create lists of tokens

device = "cuda"
trn = [train_tokens["input_ids"].to(device), train_tokens["attention_mask"].to(device), torch.tensor(train_labels).to(device)]
val = [val_tokens["input_ids"].to(device), val_tokens["attention_mask"].to(device), torch.tensor(val_labels).to(device)]

In [ ]:
# Dataloader class

BATCH_SIZE = 32
class ClassificationData(pl.LightningDataModule):
    def __init__(self, trn, val):
        super().__init__()

        self.trn = DataLoader(TensorDataset(*trn), batch_size=BATCH_SIZE)
        self.val = DataLoader(TensorDataset(*val), batch_size=BATCH_SIZE)

    def train_dataloader(self): return self.trn
    def val_dataloader(self): return self.val

dls = ClassificationData(trn, val)

In [ ]:
# This should return a list of 4 tensors - input_ids, attention_masks, token_type_ids, and labels
next(iter(dls.trn))

[tensor([[ 7100, 26949,  1251,  ..., 18936,  1544,   442],
         [ 6022,  2236,  2279,  ...,   173,  2317,   720],
         [ 3542,  2910,  9854,  ...,  1759,   685,     6],
         ...,
         [  297,   116,   406,  ...,  7743, 13997, 19391],
         [ 1341,   829,   201,  ..., 11977,  3604, 15489],
         [  994,   635,  5305,  ...,   856,   856,  7330]], device='cuda:0'),
 tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'),
 tensor([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
         0, 1, 1, 0, 0, 1, 1, 1], device='cuda:0')]

Training Text Classifier

In [ ]:
from transformers import RobertaModel
roberta_model = RobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# The RobertaClassifier class

class RobertaClassifier(pl.LightningModule):
    def __init__(self, dropout_p, hid_dim, output_dim):
        super().__init__()
        self.roberta = roberta_model
        self.dropout = torch.nn.Dropout(dropout_p)
        self.linear_1 = torch.nn.Linear(hid_dim,hid_dim)
        self.linear_2 = torch.nn.Linear(hid_dim, output_dim)
        self.loss = torch.nn.NLLLoss()

    def forward(self, input_ids, attention_mask):
        x1 = self.roberta(input_ids, attention_mask=attention_mask)[0]
        x1 = x1[:,0]
        x1 = self.dropout(torch.nn.ReLU()(self.linear_1(x1)))
        output  = torch.log_softmax(self.linear_2(x1), dim = 1)
        return output

    def training_step(self, batch, ix):
        pred = self(batch[0], batch[1])
        loss = self.loss(pred, batch[2].view(-1))
        return loss

    def validation_step(self, batch, ix):
        pred = self(batch[0], batch[1])
        loss = self.loss(pred, batch[2].view(-1))
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

m = RobertaClassifier(0.5, 768, 2)

In [ ]:
#Training the model, dls is object of the dataloader class

device = "cuda"
t = pl.Trainer(max_epochs=1, gpus=1)
t.fit(m.to(device), dls)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type         | Params
------------------------------------------
0 | roberta  | RobertaModel | 124 M 
1 | dropout  | Dropout      | 0     
2 | linear_1 | Linear       | 590 K 
3 | linear_2 | Linear       | 1.5 K 
4 | loss     | NLLLo

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
print(m)